In [1]:

import pandas as pd
import statsmodels
import statsmodels.api as sm
from statsmodels.formula.api import ols
import tweepy
import json
import os
from collections import Counter
import numpy as np
from collections import defaultdict
from matplotlib import pyplot as plt
import time
from datetime import datetime
from dateutil.parser import parse

input_keywords="input_keywords.txt"
consumer_key="hME3af4IKB8PG1bACoFWIOsb7"
consumer_secret="5NVgAZqgezRM1fApMwuv4bF2lagForppzEClipxYymL4EnWztb"
access_token="2257981634-ERJNYPvjln32aQlNygNAyUNImFn2TCzE1VBYIkG"
access_token_secret="hqmDnz3x0iQoEbmAJvq6AAN1rFHbuFetoTs5hPf20Gqdq"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# funciton to get tweet ids already in file


public_tweets = api.home_timeline()
tweets=[]
for tweet in public_tweets:
    print(tweet.text)
    tweets.append(tweet._json)
print(json.dumps(tweets[0], indent=4))


df=pd.DataFrame(tweets)
df.head()





I need a podcast where i just rant incoherently about what's on my mind every week.

Then after 2 hours I can be li… https://t.co/xouYP3hoDS
Quick video about Quiplash copyright claims, we’re sorting this but if you’re affected and we’re not aware please m… https://t.co/x9r9lmegd5
I was gonna take an ominous photo, but my phone camera has better night vision than i do.

My evening is ruined.
Isolated*
I get to walk out here in peaceful bliss, looking forward to kerbal space program 2 with hopeful enthusiasm like some kind of jackass.
I think the lesson to be learned here is that I'm outside, at a quarter to 2 in the morning, and getting raped or a… https://t.co/hEd6SPrwny
But I want to play a video game about building rockets. I'm never gonna *go* there. But both imagination and games *let* me.
Maybe that's just how dumb we are, as a species. Those are actual stars up there. Orbiting them are, quite possibly… https://t.co/z3moCC38CK
I'm in a fairly area, and im outside looking up at th

,created_at,id,id_str,text,truncated,entities,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,...,favorited,retweeted,lang,possibly_sensitive,possibly_sensitive_appealable,retweeted_status,quoted_status_id,quoted_status_id_str,quoted_status,extended_entities
0,Tue Apr 21 00:03:49 +0000 2020,1252387528759369731,1252387528759369731,I need a podcast where i just rant incoherentl...,True,"{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/android"" ...",NaN,None,NaN,...,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Tue Apr 21 00:03:30 +0000 2020,1252387446534234113,1252387446534234113,"Quick video about Quiplash copyright claims, w...",True,"{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,NaN,...,False,False,en,False,False,NaN,NaN,NaN,NaN,NaN
2,Mon Apr 20 23:55:28 +0000 2020,1252385425823395842,1252385425823395842,"I was gonna take an ominous photo, but my phon...",False,"{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/android"" ...",NaN,None,NaN,...,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Mon Apr 20 23:53:18 +0000 2020,1252384880056365068,1252384880056365068,Isolated*,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/android"" ...",1.252385e+18,1252384614531792897,878196469.0,...,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Mon Apr 20 23:52:16 +0000 2020,1252384622366740481,1252384622366740481,"I get to walk out here in peaceful bliss, look...",False,"{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/android"" ...",1.252385e+18,1252384620504440834,878196469.0,...,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
def get_ids(file_name):
    result=[]
    try:
        with open(file_name, 'r') as fp:
            line=fp.readline()
            while line:
                tjson=json.loads(line)
                result.append(tjson["id"])
                line=fp.readline()
        return result
    except IOError:
        return result

# call get_ids on starting file
tweet_ids = get_ids('tweets.json')

# function from tutorial
def crawl_tweet(file_name="raw_tweet.json", search_key="Twitter", max_tweet=1000, max_id=-1):
    tweet_count = 0
    # append to file instead of writing so we don't wipe previous data
    with open(file_name, 'a') as fp:
        fp.write("")
    while tweet_count<max_tweet:
        try:
            if max_id<=0:
                new_tweets=api.search(q=search_key, count=100)
            else:
                new_tweets=api.search(q=search_key, count=100, max_id=str(max_id-1))
            if not new_tweets:
                print('No more tweets')
                break
            with open(file_name, 'a') as fp:
                for tweet in new_tweets:
                    # convert tweet to json
                    tjson=tweet._json
                    # filter duplicate tweets
                    if tjson['id_str'] not in tweet_ids:
                        # add tweet to text file
                        fp.write(json.dumps(tjson)+"\n")
                        # add tweet id to tweet_ids
                        tweet_ids.append(tjson['id_str'])
            max_id=new_tweets[-1].id
            tweet_count+=len(new_tweets)
            print(f"Number of tweets: {tweet_count}")
        except tweepy.TweepError as e:
            print("some error: " + str(e))
            break

In [3]:
### read keywords

input_keywords="input_keywords.txt"
# read keyswords in
with open(input_keywords) as fp2:
    keywords = fp2.readlines()
    
# remove newlines
for i in range(len(keywords)):
    keywords[i] = keywords[i].replace('\n', '')


In [4]:

# call crawel tweets on each keyword
for k in keywords:
    crawl_tweet(file_name='tweets.json', search_key=k, max_tweet=100)

Number of tweets: 100
Number of tweets: 100
Number of tweets: 100
Number of tweets: 100


In [5]:
### crawl tweets from oldest stored

# call get_ids on starting file
tweet_ids = get_ids('tweets.json')
last = min(tweet_ids)
# call crawel tweets on each keyword
for k in keywords:
    crawl_tweet('tweets.json', k, 100/len(keywords), last)

Number of tweets: 100
Number of tweets: 100
Number of tweets: 100
Number of tweets: 100


In [6]:
# load tweets.json file
def load_json(file_name):
    result=[]
    with open(file_name, 'r') as fp:
        line=fp.readline()
        while line:
            tjson=json.loads(line) #decode json
            result.append({
                 "Date":tjson["created_at"],
                    "id":tjson["id_str"],
                 "user_name":tjson["user"]["name"],
                   
                    "favorite_count":tjson["favorite_count"],#number of favorites
                    "retweet_count":tjson["retweet_count"],#number of retweets
                    "user_id_str":tjson["user"]["id_str"],
                
                
            })

            line=fp.readline()
    return result
data_list=load_json("tweets.json")
df=pd.DataFrame(data=data_list)
#df = df.drop_duplicates(subset=['id'], keep=False)

df

,Date,id,user_name,favorite_count,retweet_count,user_id_str
0,Sun Apr 19 21:16:49 +0000 2020,1251983111904268291,(((GloriaV))),0,7812,87273880
1,Sun Apr 19 21:16:41 +0000 2020,1251983079389958144,trvestuff,0,1,1136477917939027968
2,Sun Apr 19 21:16:38 +0000 2020,1251983065703870465,Mattie Gage,0,1,828053121968529411
3,Sun Apr 19 21:16:37 +0000 2020,1251983062843502592,albert ratcliffe,0,0,1082111299
4,Sun Apr 19 21:16:34 +0000 2020,1251983049396563969,trvestuff,0,1,1136477917939027968
...,...,...,...,...,...,...
3941,Sun Apr 19 10:00:00 +0000 2020,1251812788244267011,Totally Vegan Buzz,1,1,997517273672937472
3942,Sun Apr 19 09:57:24 +0000 2020,1251812130967433217,katie jenkins,0,43,2793514363
3943,Sun Apr 19 09:56:57 +0000 2020,1251812019197657090,eggsy,0,43,3332336051
3944,Sun Apr 19 09:56:27 +0000 2020,1251811894563950592,Wendy O'Mara #FBPE,0,43,332259447


In [7]:
#converting date into more readable form 
Date_list= df["Date"].values
Date_list
for date in Date_list:
#     print('Parsing: ' + date)
    dt = parse(date)
    print(dt.date())
df["Date"]=dt.date()
df
   

2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19
2020-04-19

,Date,id,user_name,favorite_count,retweet_count,user_id_str
0,2020-04-19,1251983111904268291,(((GloriaV))),0,7812,87273880
1,2020-04-19,1251983079389958144,trvestuff,0,1,1136477917939027968
2,2020-04-19,1251983065703870465,Mattie Gage,0,1,828053121968529411
3,2020-04-19,1251983062843502592,albert ratcliffe,0,0,1082111299
4,2020-04-19,1251983049396563969,trvestuff,0,1,1136477917939027968
...,...,...,...,...,...,...
3941,2020-04-19,1251812788244267011,Totally Vegan Buzz,1,1,997517273672937472
3942,2020-04-19,1251812130967433217,katie jenkins,0,43,2793514363
3943,2020-04-19,1251812019197657090,eggsy,0,43,3332336051
3944,2020-04-19,1251811894563950592,Wendy O'Mara #FBPE,0,43,332259447


In [12]:

#groupby date
group=df.groupby(by=["Date"])
new_df=group.aggregate({
    "id":["count"],
    "favorite_count":["sum"],
    "retweet_count":["sum"]
})
new_df.rename(columns={"id_str":"tweet_count"}, inplace=True)
print(new_df)
new_df.to_csv("climate_data.csv", index=True)


              id favorite_count retweet_count
           count            sum           sum
Date                                         
2020-04-19  3946           3082       6249115
